In [1]:
import os
import numpy as np
import tensorflow as tf

from tensorflow import keras

In [2]:
def get_data(proper_name, improper_name):
    with open(os.path.join('datasets', proper_name), 'r') as f:
        proper_reber_lines = [line.rstrip() for line in f]

    with open(os.path.join('datasets', improper_name), 'r') as f:
        improper_reber_lines = [line.rstrip() for line in f]

    max_len = len(max([max(proper_reber_lines, key=len), max(improper_reber_lines, key=len)], key=len))
    proper_reber_lines = keras.preprocessing.sequence.pad_sequences(
        [[ord(char) for char in line] for line in proper_reber_lines],
        maxlen=max_len)
    improper_reber_lines = keras.preprocessing.sequence.pad_sequences(
        [[ord(char) for char in line] for line in improper_reber_lines],
        maxlen=max_len)

    return proper_reber_lines, improper_reber_lines, max_len


proper_reber_lines, improper_reber_lines, max_len = get_data('proper_reber.txt', 'improper_reber.txt')

In [3]:
def get_arrays(proper_reber_lines, improper_reber_lines):
    truths = np.ones((proper_reber_lines.shape[0], 1))
    truth_data = np.hstack([proper_reber_lines, truths])
    falses = np.zeros((improper_reber_lines.shape[0], 1))
    false_data = np.hstack([improper_reber_lines, falses])
    all_data = np.vstack([truth_data, false_data])

    np.random.seed(123)
    np.random.shuffle(all_data)

    eighty_percent_count = int(.8 * all_data.shape[0])
    train = all_data[:eighty_percent_count, :]
    twenty_percent_count = int(.1 * train.shape[0])
    valid = train[:twenty_percent_count, :]
    train = train[twenty_percent_count:, :]
    test = all_data[eighty_percent_count:, :]

    return train, valid, test


train, valid, test = get_arrays(proper_reber_lines, improper_reber_lines)

Looking for a classic-ML baseline model

In [4]:
from sklearn.linear_model import LogisticRegressionCV
from sklearn.svm import LinearSVC, SVC
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
import xgboost as xgb

In [5]:
pipeline = make_pipeline(StandardScaler(), LogisticRegressionCV(cv=5, max_iter=1000))

pipeline.fit(train[:, :-1], train[:, -1])

pipeline.score(train[:, :-1], train[:, -1])

0.8624172762103797

In [6]:
lin_svc_grid = make_pipeline(StandardScaler(), LinearSVC(C=.10, max_iter=10000))

lin_svc_grid.fit(train[:, :-1], train[:, -1])

lin_svc_grid.score(train[:, :-1], train[:, -1])

0.8498780912574017

In [7]:
params = {
    'classifier__C': [.01, 1, 2, 4],
    'classifier__degree': [2, 3, 4],
    'classifier__kernel': ['poly', 'rbf', 'sigmoid'],
    'classifier__gamma': ['scale', 'auto']
}
svc_pipeline = Pipeline(
    steps=[
        ('scaler', StandardScaler()),
        ('classifier', SVC())
    ])
svc_search = GridSearchCV(svc_pipeline, params)

svc_search.fit(train[:, :-1], train[:, -1])

svc_search.score(train[:, :-1], train[:, -1])

0.8760013932427726

In [8]:
model_path = os.path.join('saved_models', 'xgb_basic_reber.json')


def find_best_tree_model(save_path: str, train_data: np.array) -> Pipeline:
    if os.path.exists(save_path):
        xgb_model = xgb.XGBRFClassifier()
        scaler = StandardScaler()

        scaler.fit(train_data[:, :-1], train_data[:, -1])
        xgb_model.load_model(save_path)

        return Pipeline(
            steps=[
                ('scaler', scaler),
                ('classifier', xgb_model)
            ])
    else:
        params = {
            'classifier__n_estimators': list(range(10, 200, 10)),
            'classifier__max_depth': [2, 3, 4],
            'classifier__max_leaves': list(range(10, 200, 10)),
            'classifier__grow_policy': ['depthwise', 'lossguide'],
            'classifier__objective': ['binary:logistic', 'binary:hinge']
        }
        xgb_search_pipeline = Pipeline(
            steps=[
                ('scaler', StandardScaler()),
                ('classifier', xgb.XGBRFClassifier())
            ])
        xgb_search = GridSearchCV(xgb_search_pipeline, params)

        xgb_search.fit(train_data[:, :-1], train_data[:, -1])
        xgb_search.best_estimator_.named_steps['classifier'].save_model(save_path)

        return Pipeline(
            steps=[
                ('scaler', xgb_search.best_estimator_.named_steps['scaler']),
                ('classifier', xgb_search.best_estimator_.named_steps['classifier'])
            ])

In [9]:
xgb_pipeline = find_best_tree_model(model_path, train)

print(f'train evaluation: {xgb_pipeline.score(train[:, :-1], train[:, -1])}')
print(f'test evaluation: {xgb_pipeline.score(test[:, :-1], test[:, -1])}')

train evaluation: 0.9216300940438872
test evaluation: 0.9160401002506265


Using datasets in this case is just nonsensical, but I decided to go with them anyway - for practice.

In [5]:
def create_target(batch):
    X = batch[:, :-1]
    Y = batch[:, -1:]

    return X, Y


def get_ds(array: np.ndarray) -> tf.data.Dataset:
    return tf.data.Dataset\
        .from_tensor_slices(array)\
        .shuffle(1000)\
        .batch(128)\
        .map(create_target)\
        .prefetch(1)


train_ds = get_ds(train)
valid_ds = get_ds(valid)
test_ds = get_ds(test)

In [8]:
model_dir = os.path.join(os.curdir, 'saved_models')


def run_nn_model(
        train_dataset: tf.data.Dataset,
        valid_dataset: tf.data.Dataset,
        max_length: int,
        in_model: keras.Model = None,
        dir_name='runs_dense',
        model_name='runs_dense',
        patience=20) -> (keras.Sequential, int):

    run_logdir_root = os.path.join(os.curdir, 'tensor_logs')
    dirs_count = len([
        name
        for name in os.listdir(run_logdir_root)
        if os.path.isdir(os.path.join(run_logdir_root, name)) and name.startswith(dir_name)
    ])
    run_logdir = os.path.join(run_logdir_root, f'{dir_name}_{dirs_count}')
    early_stopping = keras.callbacks.EarlyStopping(monitor='val_loss', patience=patience)
    model_checkpoint = keras.callbacks.ModelCheckpoint(os.path.join(model_dir, f'{model_name}_{dirs_count}.h5'), save_best_only=True)
    tensorboard = keras.callbacks.TensorBoard(run_logdir, histogram_freq=1, profile_batch=10)

    if in_model is None:
        in_model = keras.Sequential([
            keras.layers.Input(shape=(max_length,)),
            keras.layers.BatchNormalization(),
            keras.layers.Dense(4 * max_length, activation='relu'),
            keras.layers.BatchNormalization(),
            keras.layers.Dropout(.4),
            keras.layers.Dense(4 * max_length, activation='relu'),
            keras.layers.BatchNormalization(),
            keras.layers.Dropout(.4),
            keras.layers.Dense(4 * max_length, activation='relu'),
            keras.layers.BatchNormalization(),
            keras.layers.Dropout(.3),
            keras.layers.Dense(1, activation='sigmoid'),
        ])

    in_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    in_model.fit(train_dataset, validation_data=valid_dataset, epochs=200, callbacks=[early_stopping, model_checkpoint, tensorboard])

    return in_model, dirs_count

In [12]:
_, dirs_count = run_nn_model(train_ds, valid_ds, max_len)
model = keras.models.load_model(os.path.join(model_dir, f'runs_dense_{dirs_count}.h5'))

model.evaluate(test_ds)

Epoch 1/200
23/23 [==============================] - 4s 32ms/step - loss: 0.6692 - accuracy: 0.6959 - val_loss: 1.1949 - val_accuracy: 0.6144
Epoch 2/200
23/23 [==============================] - 0s 18ms/step - loss: 0.5452 - accuracy: 0.7604 - val_loss: 0.8059 - val_accuracy: 0.6207
Epoch 3/200
23/23 [==============================] - 0s 17ms/step - loss: 0.4767 - accuracy: 0.7865 - val_loss: 0.7637 - val_accuracy: 0.6207
Epoch 4/200
23/23 [==============================] - 0s 16ms/step - loss: 0.4439 - accuracy: 0.8137 - val_loss: 0.7615 - val_accuracy: 0.6426
Epoch 5/200
23/23 [==============================] - 0s 16ms/step - loss: 0.4398 - accuracy: 0.8161 - val_loss: 0.7145 - val_accuracy: 0.6834
Epoch 6/200
23/23 [==============================] - 0s 17ms/step - loss: 0.4354 - accuracy: 0.8147 - val_loss: 0.6397 - val_accuracy: 0.7335
Epoch 7/200
23/23 [==============================] - 0s 15ms/step - loss: 0.4017 - accuracy: 0.8265 - val_loss: 0.6921 - val_accuracy: 0.7304
Epoch 

[0.19312109053134918, 0.9348371028900146]

In [13]:
model_numbers = list(range(22, 32, 1))
best_result = 0
best_dense_model = None

for number in model_numbers:
    try:
        name = f'runs_dense_{number}.h5'
        tmp_model = keras.models.load_model(os.path.join(model_dir, name))
        tmp_result = tmp_model.evaluate(test_ds)[1]

        if tmp_result > best_result:
            best_result = tmp_result
            best_dense_model = tmp_model
    except Exception as exc:
        print(exc)
        print('=========================================================')

No file or directory found at .\saved_models\runs_dense_22.h5
No file or directory found at .\saved_models\runs_dense_23.h5
No file or directory found at .\saved_models\runs_dense_24.h5
No file or directory found at .\saved_models\runs_dense_25.h5
No file or directory found at .\saved_models\runs_dense_26.h5
No file or directory found at .\saved_models\runs_dense_27.h5
No file or directory found at .\saved_models\runs_dense_28.h5
No file or directory found at .\saved_models\runs_dense_29.h5
No file or directory found at .\saved_models\runs_dense_30.h5
No file or directory found at .\saved_models\runs_dense_31.h5


Now the fun part - embedded reber grammars

In [11]:
proper_embedded_reber_lines, improper_embedded_reber_lines, max_len_embed = get_data('proper_embedded_reber.txt', 'improper_embedded_reber.txt')
train_embed, valid_embed, test_embed = get_arrays(proper_embedded_reber_lines, improper_embedded_reber_lines)

In [15]:
embed_model_path = os.path.join('saved_models', 'xgb_embed_reber.json')
embed_xbg_pipeline = find_best_tree_model(embed_model_path, train_embed)

In [16]:
print(f'train evaluation: {embed_xbg_pipeline.score(train_embed[:, :-1], train_embed[:, -1])}')
print(f'test evaluation: {embed_xbg_pipeline.score(test_embed[:, :-1], test_embed[:, -1])}')

train evaluation: 0.9503472222222222
test evaluation: 0.955


In [12]:
train_ds_embed = get_ds(train_embed)
valid_ds_embed = get_ds(valid_embed)
test_ds_embed = get_ds(test_embed)

In [18]:
model_embed, dirs_count = run_nn_model(
    train_ds_embed,
    valid_ds_embed,
    None,
    keras.Sequential([
        keras.layers.Input(shape=(max_len_embed,)),
        keras.layers.BatchNormalization(),
        keras.layers.Dense(4 * max_len_embed, activation='relu'),
        keras.layers.BatchNormalization(),
        keras.layers.Dropout(.4),
        keras.layers.Dense(4 * max_len_embed, activation='relu'),
        keras.layers.BatchNormalization(),
        keras.layers.Dropout(.4),
        keras.layers.Dense(4 * max_len_embed, activation='relu'),
        keras.layers.BatchNormalization(),
        keras.layers.Dropout(.3),
        keras.layers.Dense(1, activation='sigmoid'),
    ]))

Epoch 1/200
23/23 [==============================] - 2s 31ms/step - loss: 0.5317 - accuracy: 0.7677 - val_loss: 0.8122 - val_accuracy: 0.5094
Epoch 2/200
23/23 [==============================] - 0s 16ms/step - loss: 0.3669 - accuracy: 0.8611 - val_loss: 0.8559 - val_accuracy: 0.5094
Epoch 3/200
23/23 [==============================] - 0s 18ms/step - loss: 0.3180 - accuracy: 0.8854 - val_loss: 0.7638 - val_accuracy: 0.5156
Epoch 4/200
23/23 [==============================] - 0s 18ms/step - loss: 0.2909 - accuracy: 0.8920 - val_loss: 0.7252 - val_accuracy: 0.5437
Epoch 5/200
23/23 [==============================] - 0s 18ms/step - loss: 0.2648 - accuracy: 0.9028 - val_loss: 0.5644 - val_accuracy: 0.6562
Epoch 6/200
23/23 [==============================] - 0s 19ms/step - loss: 0.2529 - accuracy: 0.9122 - val_loss: 0.4966 - val_accuracy: 0.7125
Epoch 7/200
23/23 [==============================] - 0s 18ms/step - loss: 0.2554 - accuracy: 0.9066 - val_loss: 0.4524 - val_accuracy: 0.7563
Epoch 

In [19]:
model_embed.evaluate(test_ds_embed)

7/7 [==============================] - 0s 3ms/step - loss: 0.1301 - accuracy: 0.9725


[0.13014397025108337, 0.9725000262260437]

Time to test it with LSTMs

In [20]:
train_embed_4reshape, valid_embed_4reshape, test_embed_4reshape = get_arrays(proper_embedded_reber_lines, improper_embedded_reber_lines)
train_embed_4reshape = train_embed_4reshape.reshape((train_embed_4reshape.shape[0], max_len_embed + 1, 1))
valid_embed_4reshape = valid_embed_4reshape.reshape((valid_embed_4reshape.shape[0], max_len_embed + 1, 1))
test_embed_4reshape = test_embed_4reshape.reshape((test_embed_4reshape.shape[0], max_len_embed + 1, 1))
train_ds_embed_reshaped = get_ds(train_embed_4reshape)
valid_ds_embed_reshaped = get_ds(valid_embed_4reshape)
test_ds_embed_reshaped = get_ds(test_embed_4reshape)
lstm_model, dirs_count = run_nn_model(
    train_ds_embed_reshaped,
    valid_ds_embed_reshaped,
    None,
    keras.Sequential([
        keras.layers.LSTM(4 * max_len_embed, input_shape=(max_len_embed, 1), return_sequences=True),
        keras.layers.BatchNormalization(),
        keras.layers.Dropout(.1),
        keras.layers.LSTM(4 * max_len_embed),
        keras.layers.Dropout(.3),
        keras.layers.Dense(1, activation='sigmoid'),
    ]),
    patience=40,
    dir_name='runs_lstm',
    model_name='runs_lstm')

Epoch 1/200
23/23 [==============================] - 8s 109ms/step - loss: 0.6926 - accuracy: 0.5913 - val_loss: 0.6683 - val_accuracy: 0.5031
Epoch 2/200
23/23 [==============================] - 1s 41ms/step - loss: 0.4968 - accuracy: 0.7691 - val_loss: 0.6525 - val_accuracy: 0.5031
Epoch 3/200
23/23 [==============================] - 1s 39ms/step - loss: 0.3431 - accuracy: 0.8569 - val_loss: 0.7518 - val_accuracy: 0.5031
Epoch 4/200
23/23 [==============================] - 1s 38ms/step - loss: 0.2678 - accuracy: 0.9028 - val_loss: 0.8774 - val_accuracy: 0.5000
Epoch 5/200
23/23 [==============================] - 1s 41ms/step - loss: 0.2966 - accuracy: 0.8847 - val_loss: 0.4949 - val_accuracy: 0.6906
Epoch 6/200
23/23 [==============================] - 1s 38ms/step - loss: 0.2593 - accuracy: 0.9118 - val_loss: 0.8069 - val_accuracy: 0.5031
Epoch 7/200
23/23 [==============================] - 1s 40ms/step - loss: 0.1534 - accuracy: 0.9538 - val_loss: 1.1221 - val_accuracy: 0.5031
Epoch

In [21]:
lstm_model.evaluate(test_ds_embed_reshaped)

7/7 [==============================] - 0s 7ms/step - loss: 0.0290 - accuracy: 0.9950


[0.028996694833040237, 0.9950000047683716]

In [22]:
gru_model, dirs_count = run_nn_model(
    train_ds_embed_reshaped,
    valid_ds_embed_reshaped,
    None,
    keras.Sequential([
        keras.layers.GRU(4 * max_len_embed, input_shape=(max_len_embed, 1), return_sequences=True),
        keras.layers.BatchNormalization(),
        keras.layers.Dropout(.1),
        keras.layers.GRU(4 * max_len_embed),
        keras.layers.Dropout(.3),
        keras.layers.Dense(1, activation='sigmoid'),
    ]),
    patience=40,
    dir_name='runs_gru',
    model_name='runs_gru')

Epoch 1/200
23/23 [==============================] - 4s 60ms/step - loss: 0.8373 - accuracy: 0.5514 - val_loss: 0.9604 - val_accuracy: 0.5031
Epoch 2/200
23/23 [==============================] - 1s 24ms/step - loss: 0.5296 - accuracy: 0.7233 - val_loss: 1.3630 - val_accuracy: 0.5031
Epoch 3/200
23/23 [==============================] - 1s 23ms/step - loss: 0.3243 - accuracy: 0.8712 - val_loss: 1.9344 - val_accuracy: 0.5031
Epoch 4/200
23/23 [==============================] - 1s 23ms/step - loss: 0.2319 - accuracy: 0.9222 - val_loss: 2.5602 - val_accuracy: 0.5031
Epoch 5/200
23/23 [==============================] - 1s 23ms/step - loss: 0.1815 - accuracy: 0.9451 - val_loss: 2.9685 - val_accuracy: 0.5031
Epoch 6/200
23/23 [==============================] - 1s 23ms/step - loss: 0.1526 - accuracy: 0.9535 - val_loss: 3.1415 - val_accuracy: 0.5031
Epoch 7/200
23/23 [==============================] - 1s 23ms/step - loss: 0.1379 - accuracy: 0.9635 - val_loss: 3.3133 - val_accuracy: 0.5031
Epoch 

In [23]:
gru_model.evaluate(test_ds_embed_reshaped)

7/7 [==============================] - 0s 8ms/step - loss: 1.2742 - accuracy: 0.6837


[1.2741901874542236, 0.6837499737739563]

Those models were built without embedding, let's start from the beginning and try out embeddings

In [28]:
POSSIBLE_CHARS = 'BEPSTVX'


def shuffle(vals: tf.RaggedTensor, targets: tf.Tensor) -> (tf.RaggedTensor, tf.Tensor):
    a = tf.random.shuffle(tf.range(vals.shape[0]))
    b = tf.reshape(a, (vals.shape[0], 1))
    shuffled_vals = tf.gather_nd(vals, b)
    shuffled_targets = tf.gather_nd(targets, b)

    return shuffled_vals, shuffled_targets


def get_data_4_embeddings(proper_name, improper_name) -> (tf.RaggedTensor, tf.Tensor):
    def string_to_ids(s, chars=POSSIBLE_CHARS):
        ids = []

        for char in s:
            try:
                ids.append(chars.index(char))
            except:
                ids.append(-1)

        return ids

    with open(os.path.join('datasets', proper_name), 'r') as f:
        raw_proper_reber_lines = [line.rstrip() for line in f]

    with open(os.path.join('datasets', improper_name), 'r') as f:
        raw_improper_reber_lines = [line.rstrip() for line in f]

    proper_lines = tf.ragged.constant([string_to_ids(s) for s in raw_proper_reber_lines])
    improper_lines = tf.ragged.constant([string_to_ids(s) for s in raw_improper_reber_lines])
    X = tf.concat([proper_lines, improper_lines], axis=0)
    y = tf.constant([[1.] for _ in range(len(raw_proper_reber_lines))] +
                 [[0.] for _ in range(len(raw_improper_reber_lines))])

    return shuffle(X, y)


X, y = get_data_4_embeddings('proper_embedded_reber.txt', 'improper_embedded_reber.txt')
X_train, y_train = X[:3000], y[:3000]
X_valid, y_valid = X[3000:], y[3000:]

In [29]:
def run_nn_model(
        X_train: tf.RaggedTensor,
        y_train: tf.Tensor,
        X_valid: tf.RaggedTensor,
        y_valid: tf.Tensor,
        in_model: keras.Model = None,
        model_name='runs_rnn',
        patience=20) -> keras.Sequential:

    run_logdir_root = os.path.join(os.curdir, 'tensor_logs')
    dirs_count = len([
        name
        for name in os.listdir(run_logdir_root)
        if os.path.isdir(os.path.join(run_logdir_root, name)) and name.startswith(model_name)
    ])
    run_logdir = os.path.join(run_logdir_root, f'{model_name}_{dirs_count}')
    early_stopping = keras.callbacks.EarlyStopping(monitor='val_loss', patience=patience)
    model_checkpoint = keras.callbacks.ModelCheckpoint(os.path.join(model_dir, f'{model_name}_{dirs_count}.h5'), save_best_only=True)
    tensorboard = keras.callbacks.TensorBoard(run_logdir, histogram_freq=1, profile_batch=10)

    in_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    in_model.fit(X_train, y_train, validation_data=(X_valid, y_valid), epochs=200, callbacks=[early_stopping, model_checkpoint, tensorboard])

    return in_model

In [30]:
model = keras.models.Sequential([
    keras.layers.InputLayer(input_shape=[None], dtype=tf.int32, ragged=True),
    keras.layers.Embedding(input_dim=len(POSSIBLE_CHARS), output_dim=5),
    keras.layers.GRU(30),
    keras.layers.Dense(1, activation="sigmoid")
])

run_nn_model(X_train, y_train, X_valid, y_valid, model)

Epoch 1/200


C:\ProgramData\Anaconda3\envs\deep_learning_with_python\lib\site-packages\tensorflow\python\framework\indexed_slices.py:444: UserWarning: Converting sparse IndexedSlices(IndexedSlices(indices=Tensor("gradient_tape/sequential_8/gru_5/RaggedToTensor/boolean_mask_1/GatherV2:0", shape=(None,), dtype=int32), values=Tensor("gradient_tape/sequential_8/gru_5/RaggedToTensor/boolean_mask/GatherV2:0", shape=(None, 5), dtype=float32), dense_shape=Tensor("gradient_tape/sequential_8/gru_5/RaggedToTensor/Shape:0", shape=(2,), dtype=int32))) to a dense Tensor of unknown shape. This may consume a large amount of memory.
  warnings.warn(


94/94 [==============================] - 3s 17ms/step - loss: 0.6430 - accuracy: 0.6697 - val_loss: 0.5299 - val_accuracy: 0.7560
Epoch 2/200
94/94 [==============================] - 1s 12ms/step - loss: 0.4003 - accuracy: 0.8120 - val_loss: 0.3520 - val_accuracy: 0.8470
Epoch 3/200
94/94 [==============================] - 1s 11ms/step - loss: 0.3157 - accuracy: 0.8657 - val_loss: 0.2923 - val_accuracy: 0.8790
Epoch 4/200
94/94 [==============================] - 1s 10ms/step - loss: 0.2779 - accuracy: 0.8983 - val_loss: 0.2388 - val_accuracy: 0.9180
Epoch 5/200
94/94 [==============================] - 1s 11ms/step - loss: 0.2293 - accuracy: 0.9223 - val_loss: 0.2227 - val_accuracy: 0.9480
Epoch 6/200
94/94 [==============================] - 1s 10ms/step - loss: 0.2077 - accuracy: 0.9357 - val_loss: 0.2589 - val_accuracy: 0.8790
Epoch 7/200
94/94 [==============================] - 1s 10ms/step - loss: 0.1897 - accuracy: 0.9457 - val_loss: 0.1600 - val_accuracy: 0.9650
Epoch 8/200
94/94 